# Novel Inertia Based Human Pose Estimation Using CNN

In [1]:
#setup
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import scipy.io as sio
from utils import *

#need to have these two lines to work on my ancient 1060 3gb
#  https://stackoverflow.com/questions/43990046/tensorflow-blas-gemm-launch-failed
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
%load_ext autoreload
%autoreload 2
%autosave 180

Autosaving every 180 seconds


In [ ]:
#Import data from MatLab SimScape Multibody Simulator

#data comes from two files
#1) n trajectories in xyz space, each length m
# traj = np.loadtxt(open("C:/Users/Derm/Desktop/traj_with_angs_1M.txt", "rb"), delimiter=",")
traj = np.loadtxt(open("simulation/data/traj_9DOF_100k.txt", "rb"), delimiter=",")

trajPts = np.shape(traj)[0] #points per trajectory
# numTraj = np.shape(traj)[1]//3 #number of total trajectories
numTraj = np.shape(traj)[1]//6 #number of total trajectories


#traj needs to be reshaped to a 3d numpy array
#as is traj[n] shows [x,y,z,x,y,z...]


#2) 7 joint angles at the end of the sequence
# jointPos = np.loadtxt(open("C:/Users/Derm/Desktop/jointPos_with_angs_1M.txt", "rb"), delimiter=",")
jointPos = np.loadtxt(open("simulation/data/jointPos_9DOF_100k.txt", "rb"), delimiter=",")

# print(traj[-1])
# print(jointPos[0])



In [ ]:
#find and fix errors in data (small number of trials had huge joint angles)
err = np.argwhere(np.abs(jointPos) > 360)
print(err)

for i in err[:,0]:
    print(jointPos[i])
    jointPos[i] = jointPos[int(np.floor(np.random.rand()*np.shape(jointPos)[0]))]

# print(jointPos[17336])

In [ ]:
#reshape traj data into 3d numpy array
# t = np.zeros([trajPts,3,numTraj]) #net 1
# for j in range(np.shape(traj)[0]):
#     for i in range(np.shape(traj)[1]//3):
#         t[j,:,i] = traj[j,3*i:3*(i+1)]
        
t = np.zeros([trajPts,6,numTraj]) #net 3
for j in range(np.shape(traj)[0]):
    for i in range(np.shape(traj)[1]//6):
        t[j,:,i] = traj[j,6*i:6*(i+1)]

        
# print(t[:,:,0]) #same as in MatLab
print(np.shape(t))
#swap axis so batch size is first axis (for TF)
t = np.swapaxes(t,0,2)
print(np.shape(t)) #[numTraj, xyz, trajPts]
#swap axis again so that conv1D moves on time and not xyz
t = np.swapaxes(t,1,2)
print(np.shape(t))

In [9]:
#Augment data by rotating about y axis

from utils import *

f1 = "simulation/data/traj_9DOF_100k.txt"
f2 = "simulation/data/jointPos_9DOF_100k.txt"
numTraj = 100000
# f1 = "simulation/data/traj_9DOF_test.txt"
# f2 = "simulation/data/jointPos_9DOF_test.txt"
# numTraj = 1000

t, jointPos = add_body_rotation(f1, f2, numTraj, mult = 1)

# print(jointPos[0])

saved augmented data as tbr, jbr


In [10]:
from network import Net4 #for 10DOF model

np.random.seed(1337)

#convert data from numpy to tensors
x_train = tf.convert_to_tensor(t,np.float32)
y_train = tf.convert_to_tensor(jointPos,np.float32)

# print(tf.shape(x_train))
# print(x_train[0,:,1])

In [11]:
model = Net4()

runLen = 30    

def scheduler(epoch, lr):
#     part1 = 2*runLen//3
#     part2 = 5*runLen//6 #net1

    part1 = runLen//3
    part2 = 2*runLen//3 #net2

    if epoch < part1:
        lr = 0.01
        return lr
    if epoch >= part1 and epoch < part2:
        lr = 0.001
        return lr
    if epoch >= part2:
        lr = 0.0001
        return lr

model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=0.001),
    loss=tf.keras.losses.MeanSquaredError(),
#     loss=tf.keras.losses.MeanAbsoluteError(),
#     metrics=[tf.keras.metrics.MeanSquaredError()],
)

summary = model.summary()
print(summary)

callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

trace = model.fit(x=x_train, y=y_train, batch_size=16, epochs=runLen, verbose=1, 
                  validation_split=0.1, callbacks = [callback], shuffle=True)


Model: "functional_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 10, 6)]      0                                            
__________________________________________________________________________________________________
batch_normalization_64 (BatchNo (None, 10, 6)        24          input_5[0][0]                    
__________________________________________________________________________________________________
conv1d_52 (Conv1D)              (None, 8, 16)        304         batch_normalization_64[0][0]     
__________________________________________________________________________________________________
conv1d_53 (Conv1D)              (None, 8, 16)        784         conv1d_52[0][0]                  
_______________________________________________________________________________________

57/57 [==============================] - 1s 14ms/step - loss: 4805.1260 - val_loss: 3480.8782
Epoch 2/30
57/57 [==============================] - 1s 10ms/step - loss: 2266.6873 - val_loss: 2472.3550
Epoch 3/30
57/57 [==============================] - 1s 10ms/step - loss: 2120.8977 - val_loss: 2222.3364
Epoch 4/30
57/57 [==============================] - 1s 10ms/step - loss: 2094.7917 - val_loss: 2189.7822
Epoch 5/30
57/57 [==============================] - 1s 10ms/step - loss: 2079.5908 - val_loss: 3517.1506
Epoch 6/30
57/57 [==============================] - 1s 10ms/step - loss: 2053.8657 - val_loss: 2204.0220
Epoch 7/30
57/57 [==============================] - 1s 10ms/step - loss: 2007.6927 - val_loss: 2090.2620
Epoch 8/30
57/57 [==============================] - 1s 10ms/step - loss: 1893.6531 - val_loss: 2326.5876
Epoch 9/30
57/57 [==============================] - 1s 10ms/step - loss: 1989.3796 - val_loss: 2291.5791
Epoch 10/30
57/57 [==============================] - 1s 10ms/step 

In [ ]:
plt.subplot(2, 1, 1)
plt.plot(trace.history['loss'], '-')
plt.plot(trace.history['val_loss'], '-')
plt.legend(['train', 'val'], loc='upper left')
plt.xlabel('iteration')
plt.ylabel('loss')
# plt.ylim(10,1000)

In [ ]:
#load larger test dataset (for calculating accuracy of network)

ft1 = "simulation/data/traj_9DOF_1k.txt"
ft2 = "simulation/data/jointPos_9DOF_1k.txt"
numTraj = 1000 #number of trajectories given in base file

tTest, jointPosTest = add_body_rotation(ft1, ft2, numTraj, mult =1)

In [ ]:
#load single test case (for use with viz)

ft1 = "simulation/data/traj_9DOF_1.txt"
ft2 = "simulation/data/jointPos_9DOF_1.txt"
ft3 = "simulation/data/jointPath.txt"
numTraj = 1 #number of trajectories given in base file

tTest, jointPosTest = add_body_rotation(ft1, ft2, numTraj, mult =1, actual_traj=ft3)

# print(tTest)
# print(jointPosTest) #issue when only one element in jointPos input file

In [ ]:
#test model- IMPORTANT TO USE NEVER BEFORE SEEN DATA

x_test = tf.convert_to_tensor(tTest,np.float32)
y_test = tf.convert_to_tensor(jointPosTest,np.float32)

prediction = model.predict(x_test)
error = abs(y_test - prediction)

#average error for estimates for each joint
avg = np.average(error,axis=0)
print("average error = ", avg)

#range for each joint:
ranges = [50, 60, 67.5, 110, 120, 360, 130, 180, 110, 360]
rel_error = avg/ranges
print("error as frac of joint range = ", np.floor(rel_error*1000)/1000) #1 is full range of joint
print("total error = ",sum(rel_error))

print("prediction: ",prediction[0])
print("actual: ", y_test[0])

np.save("simulation/data/prediction.npy", prediction)

Best total error: 

1.52 @ 10k  dataset, mult 1, BS 128, MSE

1.29 @ 100k dataset, mult 2, BS 128, MSE

1.15 @ 100k dataset, mult 1, BS 128, MSE

1.10 @ 100k dataset, mult 1, BS 64,  MSE, Val loss = 851, 10 epoch

0.97 @ 100k dataset, mult 1, BS 128,  MSE, Val loss = 623, 30 epoch

0.87 @ 100k dataset, mult 1, BS 128,  MSE, Val loss = 556, 100 epoch


Notes:
artifially creating more data by rotating about y axis at different angles is a bad idea. This teaches the network to memorize patterns of movement rather than learn the underlying dynamic models.

In [ ]:
model.save("10DOF.kmod")

In [ ]:
#load best scoring model

model = tf.keras.models.load_model("10DOF.kmod")

In [ ]:
#proof my model is doing better than completely random guessing

np.random.seed(None)

# print(actual)
# print(tf.shape(actual)) #[99 7]
B = tf.random.uniform([1000,7])

# B = tf.ones([99,7])
B = B *tf.constant([25., 30., 33.75, 55. , 60., 180., 65.]) + tf.constant([0., 0., 26.25, -35., 30., 0., -65.])

# print(tf.shape(B))
# print(tf.shape(actual))

fake_error = (y_test - B)
# print(fake_error)

fake_avg = tf.math.reduce_mean(tf.math.abs(fake_error), axis=0)
print(fake_avg)

rel_fake_error = fake_avg/ranges

print("error as frac of joint range: ",rel_fake_error)
print("total error: ", sum(rel_fake_error))

#NOTE: these are not all the same becuase the starting ranges for joint positions do NOT fall in the middle of all
#      possible positions for each joint

In [ ]:
from scipy.spatial.transform import Rotation as R
r = R.from_euler('zyx', [
[90, 0, 0],
[0, 45, 0],
[45, 60, 30]], degrees=True)

print(r)
r.as_quat()

v = [1, 2, 3]

r.apply(v)

In [ ]:
r1 = R.from_euler('z', 90, degrees=True)
r2 = R.from_euler('x', 90, degrees=True)
r3 = r1*r2

v = [1, 2, 3]

rot1 = r1.apply(v)
rot2 = r3.apply(v)
print(rot2)


In [ ]:
a, b, c= r3.apply([1, 2, 3])
print(a,b,c)
